# 深層学習

*Deep Learning*

深層学習を用いて言語モデルを作成する。  
ニューラルネットワークでマルコフモデルを作成し、深層学習における言語モデル実装の基礎を学ぶ。


---

## データセット

wiki40b

In [1]:
import tensorflow_datasets as tfds
ds = tfds.load('wiki40b/ja', split='test')
ds = list(ds.as_numpy_iterator())

2023-06-18 23:58:37.018815: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 23:58:37.515817: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-18 23:58:38.148164: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-18 23:58:38.168463: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the requir

In [2]:
data = []
for sample in ds:
    text = sample['text'].decode()
    sections = text.split('_START_SECTION_')
    for section in sections[1:]:
        sentence = section.split('_START_PARAGRAPH_')[1]
        sentence = sentence.replace('_NEWLINE_', '')
        sentence = sentence.replace('\n', '')
        data.append(sentence)

print('num of data:', len(data))
data[:5] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤


---

## 前処理

テキストをNNで扱える形に変換する。

### トークン化

文章をトークンごとに分割する。トークンとは文章を構成する最小の単位で、単語や句読点などが該当する。  
トークン化は分かち書きと似た意味であるが、言語モデルの領域で最小単位をトークンと呼ぶことが一般的なことや、単純にトークン化と呼ぶことが多いことから、本節以降トークン化と呼ぶ。

トークン化には、分かち書き同様形態素解析を用いる。

In [3]:
import MeCab

各文章をトークンのリストとして格納する。

In [4]:
data_tokens = []
tagger = MeCab.Tagger('-Owakati')
for sentence in data:
    tokens = tagger.parse(sentence).strip().split(' ') # トークンのリスト
    data_tokens.append(tokens)

data_tokens[0][:10] # example

['「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」', '日本']

### ID化

NNでは文字列を扱えないので、トークン1つ1つにIDを割り当てる。

<br>

torchtextの`vocab`モジュールを使用する。

[torchtext.vocab — Torchtext 0.15.0 documentation](https://pytorch.org/text/stable/vocab.html#)

In [5]:
from torchtext.vocab import vocab, build_vocab_from_iterator

単語とその出現頻度を表した辞書を入力し、`Vocab`オブジェクトを作成する。`min_freq`で指定した出現頻度以下の単語は無視される。

In [6]:
v = vocab({'私': 4, 'りんご': 1, '食べる': 2, '好き': 3}, min_freq=1)
v(['私', 'りんご', '好き'])

[0, 1, 3]

`build_vocab_from_iterator()`でトークンのリスト（のリスト）から`vocab`オブジェクトを作成できる。

In [7]:
v = build_vocab_from_iterator(data)
v.get_itos()[:10] # vocabrary examples

['の', '、', 'に', 'た', '。', 'る', 'は', 'ー', 'と', 'を']

これを使ってトークン列をID列に変換する。

In [9]:
data_ids = []
v = build_vocab_from_iterator(data_tokens)
for tokens in data_tokens:
    data_ids.append(v(tokens))

print('num of vocabrary:', len(v))
data_ids[0][:10] # example

num of vocabrary: 259324


[21, 4305, 511, 2, 4, 4751, 68960, 44, 22, 55]